In [1]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [75]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [76]:
%%capture pwd
!pwd

In [77]:
# uid = "2021-05-05-which-firms-and-industries-have-been-most-affected-by-covid-update"  # article unique ID
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
height, width

(300, 500)

# Fig 1

In [157]:
df = (
    pd.read_excel(
        "raw/Figures.xlsx",
        skiprows=1,
        nrows=3,
    )
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
df = (
    df.set_index("Unnamed: 0")
    .stack()
    .reset_index()
    .set_index(["level_1", "Unnamed: 0"])
    .unstack()[0]
    .reset_index()
)
df.columns = ["year", "morbidly", "obese"]
df["year"] = df["year"].astype(str)

In [158]:
f = "fig1_obesity"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
df.head()

,year,morbidly,obese
0,1993,0.824392,14.904475
1,1994,1.015056,15.668642
2,1995,0.865891,16.444999
3,1996,0.916274,17.488879
4,1997,1.599698,18.428014


In [170]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:T",
        axis=alt.Axis(
            grid=False,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
        ),
    )
)
area1 = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.8,
    stroke=colors["eco-gray"],
    strokeWidth=0.5,
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-turquiose"], offset=0.9),
        ],
        x1=0.8,
        x2=1,
        y1=1,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "obese:Q",
        axis=alt.Axis(
            grid=True,
            title="%",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            labelAlign="left",
            ticks=False,
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=20,
            titleY=37,
            titleAngle=0,
            titleFontSize=10,
            titleFontWeight="normal",
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
    )
)
area2 = (
    base.mark_area(
        interpolate="monotone",
        fillOpacity=0.8,
        stroke=colors["eco-gray"],
        strokeWidth=0.5,
        color=alt.Gradient(
            gradient="linear",
            stops=[
                alt.GradientStop(color="white", offset=0),
                alt.GradientStop(color=colors["eco-blue"], offset=0.9),
            ],
            x1=0.8,
            x2=0.8,
            y1=0.8,
            y2=0,
        ),
    )
    .encode(y=alt.Y("stack:Q"))
    .transform_calculate(stack="datum.morbidly+datum.obese")
)
label1 = (
    alt.Chart(
        pd.DataFrame(
            [{"x": "1999-03-15", "y": 24, "t": "Obese", "c": colors["eco-turquiose"]}]
        )
    )
    .mark_text(yOffset=5, size=10, align="right")
    .encode(text="t", x="x:T", y="y:Q", color=alt.Color("c:N", scale=None))
)
label2 = (
    alt.Chart(
        pd.DataFrame(
            [
                {
                    "x": "2004-03-01",
                    "y": 26,
                    "t": "Morbidly obese",
                    "c": colors["eco-blue"],
                }
            ]
        )
    )
    .mark_text(yOffset=-5, size=10, align="right")
    .encode(text="t", x="x:T", y="y:Q", color=alt.Color("c:N", scale=None))
)
layer = (
    (area2 + area1 + label1 + label2)
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=400)
)
for j in ["json", "png", "svg"]:
    layer.save("visualisation/" + f + "." + j)
layer

Fontconfig error: Cannot load default config file
Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 2

In [190]:
df = (
    pd.read_excel(
        "raw/Figures.xlsx",
        skiprows=6,
        nrows=10,
    )
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
df = df.set_index("Category").stack().reset_index()
df.columns = ["category", "obesity", "value"]

In [248]:
f = "fig2_deprivation"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
df.head()

,category,obesity,value
0,1,Obese,27.458032
1,1,Severely obese,7.525181
2,2,Obese,26.560227
3,2,Severely obese,6.804313
4,3,Obese,25.106422


In [249]:
bars = (
    alt.Chart(f2)
    .mark_bar(size=14, opacity=0.9)
    .encode(
        y=alt.Y(
            "category:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="Index of Multiple deprivation",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleX=0,
                titleY=-5,
                titleAngle=0,
                titleFontSize=10,
                titleFontWeight="normal",
                titleAlign="left",
            ),
        ),
        color=alt.Color(
            "obesity:N",
            scale=alt.Scale(
                domain=df["obesity"].unique(),
                range=[colors["eco-turquiose"], colors["eco-blue"]],
            ),
        ),
        order="category:O",
    )
)
text = bars.mark_text(dx=-12, dy=1, color="white").encode(
    text=alt.Text("SAmmount:Q", format=".0f"),
    color=alt.condition(
        datum.SAmmount < 3,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
)
labels = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": 0.5, "y": 1, "t": "most deprived"},
                {"x": 0.5, "y": 10, "t": "least deprived"},
            ]
        )
    )
    .mark_text(yOffset=1, size=10, align="left", color="white", baseline="middle")
    .encode(text="t", x="x:Q", y="y:N")
)

layer = (
    (bars + text + labels)
    .encode(
        x=alt.X(
            "SAmmount:Q",
            stack="zero",
            axis=alt.Axis(
                grid=False,
                title="%",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleAnchor="end",
                titleY=-15,
                titleFontSize=10,
                titleFontWeight="normal",
            ),
            # scale=alt.Scale(domain=[0, 35]),
        )
    )
    .properties(height=alt.Step(20), width=300)
    .configure_legend(title=None)
    .configure_view(stroke=None)
    .transform_aggregate(SAmmount="sum(value)", groupby=["category", "obesity"])
)
for j in ["json", "png", "svg"]:
    layer.save("visualisation/" + f + "." + j)
layer

WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
Fontconfig error: Cannot load default config file
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 3

In [277]:
df = (
    pd.read_excel(
        "raw/Figures.xlsx",
        skiprows=51,
        nrows=8,
    )
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
df

,Figure 4,"Per 100,000 population - all persons6"
0,2011/12,530.4
1,2012/13,568.1
2,2013/14,709.8
3,2014/15,845.9
4,2015/16,995.5
5,2016/17,1158.6
6,2017/18,1322.8
7,2018/19,1615.0


# Fig 4

In [258]:
df = (
    pd.read_excel(
        "raw/Figures.xlsx",
        skiprows=51,
        nrows=8,
    )
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
df.columns = ["year", "value"]
df["year"] = df["year"].str.split("/").str[0].astype(str)

In [259]:
f = "fig4_hospitalisation"
f4 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4 = df
df.head()

,year,value
0,2011,530.4
1,2012,568.1
2,2013,709.8
3,2014,845.9
4,2015,995.5


In [276]:
base = alt.Chart(f4).encode(
    x=alt.X(
        "year:T",
        axis=alt.Axis(
            grid=False,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
        ),
    )
)
area1 = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.8,
    stroke=colors["eco-gray"],
    strokeWidth=0.5,
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-turquiose"], offset=0.9),
        ],
        x1=0.8,
        x2=1,
        y1=1,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "value:Q",
        axis=alt.Axis(
            grid=True,
            title="persons / 100,000 population",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            labelAlign="left",
            ticks=False,
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=30,
            titleY=45,
            titleAngle=0,
            titleFontSize=10,
            titleFontWeight="normal",
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
    )
)
layer = (
    (area1)
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=400)
)
for j in ["json", "png", "svg"]:
    layer.save("visualisation/" + f + "." + j)
layer

Fontconfig error: Cannot load default config file
Fontconfig error: Cannot load default config file


alt.Chart(...)